# Concrete Image Classification
By: Anthony Sheu

This project uses a 50/50 split dataset of positive (with cracks) and negative (without cracks) images of concrete. The data was obtained from https://data.mendeley.com/datasets/5y9wdsg2zt/2.

This binary classification model aims to be able to guess whether a provided image is positive or negative. Firstly, the images are randomly put in a 80% training, 20% validation split. Then the images will be pre-processed using transformations and finally fed through a pre-built CNN network with transfer learning to only modify the last layer (Resnet 18). The libraries used in this project are Pytorch (for deep learningm models) and torchvision (for transformations, training, and inferencing). The metrics will be tracked using WANDB and further analyzed.

## Image Processing
Below is an image of a NEGATIVE (left) and POSITIVE (right) concrete crack.

In [ ]:
from IPython.display import display, HTML

display(HTML(
"""
    <div style="display: flex; gap: 20px; text-align: center;">
    <div>
        <img src="data/train/Negative/00001.jpg" width="300"><br>
        <strong>Negative</strong>
    </div>
    <div>
        <img src="data/train/Positive/00001.jpg" width="300"><br>
        <strong>Positive</strong>
    </div>
    </div>
"""
))